By: Michał Marusiński, Jan Łucarz

"""
TO CHECK!!!
from scipy.io import loadmat
import pandas as pd

data = loadmat(r"C:\Users\19194\Desktop\Data.mat")


df = pd.DataFrame(data['pic_data'])  


df.to_csv(r"C:\Users\19194\Desktop\Classes\DW.csv", index=False)
"""

Necessary libraries

In [74]:
from scipy.io import loadmat
import pandas as pd
import numpy as np


Select **your computer path according to preprocesssed_pic_data.mat location**

In [4]:
data_pic = loadmat(r"C:\Users\48660\Desktop\Development Workshop\preprocessed\preprocessed_pic_data\preprocessed_pic_data.mat")

First--- how the matfile looks

We make dataframe out of just 1 sample (patient) and transpose it to have the data in a proper shape and format. 

In [94]:
pic_data = data_pic['pic_data']
print(f"Shape of the data: {pic_data.shape}, means it has {pic_data.shape[0]} patients with {pic_data.shape[1] - 1} columns each")
patient_01 = pd.DataFrame(pic_data[0]).transpose()
patient_01.head()


Shape of the data: (304, 6), means it has 304 patients with 5 columns each


,0,1,2,3,4,5
0,[int_2022_12_16_10-29 S01_pic_000_1_L.mat],"[[0.0], [0.001], [0.002], [0.003], [0.004], [0...","[[21.02334936534498], [21.012304013466498], [2...","[[0.0], [4.956492306396996], [9.96496093432668...","[[4.955492306396995], [9.963960934326678], [14...","[[0], [40451], [10000], [41182], [10000], [410..."


In [95]:
type(patient_01[0])

pandas.core.series.Series

Creating:
* file_name_creator for extracting proper file_name for each file
* dataframe_creator for creating clean DataFrame out of single sample of mat file

In [102]:
def file_name_creator(patient: np.ndarray):
    name_of_file = patient[0][0]
    name_of_file = name_of_file.replace('.', '_')
    file_name_split = name_of_file.split(sep = '_')
    file_name_split.pop(-1) #deleting .mat word
    file_name_split[-1] ='LEFT_EYE' if 'L' else 'RIGHT_EYE'     #switching L / R to LEFT / RIGHT EYE
    file_name = file_name_split = '_'.join(file_name_split)
    return file_name

In [112]:
def dataframe_creator(patient_data_raw: pd.Series):
    column_names = ['time', 'pupil diameter [mm]', 'trigger segmentStart', 'tigger segmentEnd', 'trigger info']
    patient_data_raw = pd.DataFrame(patient_data_raw).transpose()
    single_patient_data = (patient_data_raw[1].explode()
                    .reset_index(drop = True)
                    .to_frame(name = column_names[0]))
    single_patient_data = single_patient_data.map(lambda x: float(x))
    
    for column_idx in range(2, patient_data_raw.shape[1]):
        single_patient_data[column_names[column_idx - 1]] = (patient_data_raw[column_idx].explode()
                                        .reset_index(drop = True)
                                        .to_frame())
    single_patient_data = single_patient_data.map(lambda x: float(x))
    
    return single_patient_data

Creating loop to sort patients into their own CSV files

To pick your target folder path of where to store all the files 


**CHANGE THE DESTINATION PATH VARIABLE ASSIGNMENT BELOW**

In [61]:
dest_path = r"C:\Users\48660\Desktop\Development Workshop\preprocessed\preprocessed_pic_data"

In [62]:
#Just a warning silencer for the loop
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [117]:
#create loop for separating patients to separate files
for patient in pic_data:
    patient_df = dataframe_creator(patient_data_raw = patient)
    patient_df.to_csv(dest_path + '\\' + file_name_creator(patient), index = False)


Check if the single csv works (remember to change path of your csv file in my_path)

In [118]:
my_path = r"C:\Users\48660\Desktop\Development Workshop\preprocessed\preprocessed_pic_data\int_2022_12_19_10-46 S02_pic_000_4_LEFT_EYE"
sample_data = pd.read_csv(my_path)
sample_data.head()

,time,pupil diameter [mm],trigger segmentStart,tigger segmentEnd,trigger info
0,0.000,20.097653,0.000000,4.990216,0.0
1,0.001,20.052911,4.991216,9.999528,40492.0
2,0.002,20.008230,10.000528,15.008282,10000.0
3,0.003,19.963623,15.009282,20.015923,40592.0
4,0.004,19.919100,20.016923,25.025841,10000.0


In [119]:
sample_data.describe()

,time,pupil diameter [mm],trigger segmentStart,tigger segmentEnd,trigger info
count,400753.000000,400170.000000,81.000000,81.000000,81.000000
mean,200.376000,15.483305,200.352679,205.299544,26105.765432
std,115.687571,2.038735,117.866874,117.763202,17565.193137
min,0.000000,8.846813,0.000000,4.990216,0.000000
25%,100.188000,14.198019,100.158204,105.167466,10000.000000
50%,200.376000,15.458429,200.324365,205.333268,40041.000000
75%,300.564000,16.623380,300.500497,305.509080,40672.000000
max,400.752000,23.225197,400.777032,400.776032,99997.000000
